In [1]:
from sklearn.datasets import fetch_openml
mnist = fetch_openml('mnist_784', version=1)

In [2]:
mnist.keys()

dict_keys(['data', 'target', 'frame', 'feature_names', 'target_names', 'DESCR', 'details', 'categories', 'url'])

In [3]:
X, y = mnist["data"], mnist["target"]
X_train, X_test, y_train, y_test = X[:60000], X[60000:], y[:60000], y[60000:]


### MNIST 데이터셋으로 분류기 만들어 테스트셋에서 97% 정확도 달성하기
- KNeighborsClassifier 활용
- weights와 n_neighbors 하이퍼파라미터로 그리드 탐색 시도해보기


- 그리드 탐색 :만족할 만한 하이퍼파라미터 조합을 찾을 때 까지 수동으로 하이퍼파라미터를 조정하는 것. 

In [6]:
from sklearn.neighbors import KNeighborsClassifier

In [7]:
from sklearn.model_selection import GridSearchCV

In [8]:
param_grid =[
    {'weights': ['uniform','distance'],
     'n_neighbors' : [3],}
]

In [9]:
k_neighbor_classifier = KNeighborsClassifier()

In [10]:
grid_search = GridSearchCV(k_neighbor_classifier, param_grid, cv=2,
                          scoring='neg_mean_squared_error', return_train_score=True, 
                          n_jobs=-1                               s, verbose=4)

In [11]:
grid_search.fit(X_train,y_train)

GridSearchCV(cv=2, error_score=nan,
             estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30,
                                            metric='minkowski',
                                            metric_params=None, n_jobs=None,
                                            n_neighbors=5, p=2,
                                            weights='uniform'),
             iid='deprecated', n_jobs=8,
             param_grid=[{'n_neighbors': [3],
                          'weights': ['uniform', 'distance']}],
             pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
             scoring='neg_mean_squared_error', verbose=0)

In [12]:
grid_search.best_params_

{'n_neighbors': 3, 'weights': 'distance'}

In [15]:
grid_search.best_estimator_

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=3, p=2,
                     weights='distance')

In [17]:
cvres = grid_search.cv_results_

In [19]:
import numpy as np

In [20]:
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(np.sqrt(-mean_score),params)

0.8076921855921764 {'n_neighbors': 3, 'weights': 'uniform'}
0.7798504130066655 {'n_neighbors': 3, 'weights': 'distance'}


In [25]:
from sklearn.metrics import accuracy_score
y_pred = grid_search.predict(X_test)
accuracy_score(y_test, y_pred)

0.9717